In [7]:
import os
from typing import Dict
from matplotlib import pyplot as plt

In [13]:
current_dir = os.getcwd()+'/adjusted_buffers'
filepaths = {}
for density_dir in os.listdir(current_dir):
    for encoding_dir in os.listdir(current_dir+'/'+density_dir):
        filepaths[f'{density_dir}-{encoding_dir}'] = f'{current_dir}/{density_dir}/{encoding_dir}/timeloop-mapper.stats.txt'

In [18]:
def parse_specs_and_stats(file_content:str)->Dict[str, str]:
    """
    _summary_

    Args:
        file_content (str): _description_

    Returns:
        Dict[str, str]: _description_
    """
    data = {}
    current_level = None
    lines = file_content.split('\n')
    ln_num = 0

    while ln_num < len(lines) :
        line = lines[ln_num].strip()
        if (line):
            if line[:5] == 'Level':
                # initialise a new dictionary and skip the next few line
                current_level = lines[ln_num + 2].split(' ')[1]
                data[current_level] = {}
                ln_num += 3
            elif line[:4] == 'Area' and current_level:
                if not ('Area' in data[current_level]):
                    area = line.split(': ')
                    data[current_level]['Area'] = float(area[1].split(' ')[0])
                    ln_num += 1
            elif line[:2] == 'fJ':
                compute = float(lines[ln_num + 4].split('= ')[1])
                if 'Algo-Compute' in data:
                    data['Compute'] = compute
                else:
                    data['Algo-Compute'] = compute
                ln_num += 5
        ln_num += 1
    return data

In [19]:
eval_data = {}
for file_key, filename in filepaths.items():
    with open(filename, 'r', encoding='utf-8') as file:
        file_content = file.read()
        eval_data[file_key] = parse_specs_and_stats(file_content)

In [20]:
eval_data

{'density_0.8-run-length-encoding': {'mac': {'Area': 417.0},
  'Buffer': {'Area': 45957.1},
  'BackingStorage': {'Area': 14872.81},
  'Algo-Compute': 10990.29,
  'Compute': 13488.09},
 'density_0.8-huffman-encoding': {'mac': {'Area': 417.0},
  'Buffer': {'Area': 33663.52},
  'BackingStorage': {'Area': 1921.37},
  'Algo-Compute': 3664.49,
  'Compute': 4497.33},
 'density_0.8-bitmask': {'mac': {'Area': 417.0},
  'Buffer': {'Area': 45957.1},
  'BackingStorage': {'Area': 13925.81},
  'Algo-Compute': 10937.66,
  'Compute': 13423.49},
 'density_0.8-no-encoding': {'mac': {'Area': 417.0},
  'Buffer': {'Area': 45910.56},
  'BackingStorage': {'Area': 13619.25},
  'Algo-Compute': 11135.95,
  'Compute': 13666.85},
 'density_0.8-uncompressed-offset-pair': {'mac': {'Area': 417.0},
  'Buffer': {'Area': 47487.98},
  'BackingStorage': {'Area': 17723.04},
  'Algo-Compute': 10675.53,
  'Compute': 13101.79},
 'density_0.8-coordinate-payload': {'mac': {'Area': 417.0},
  'Buffer': {'Area': 45957.1},
  'Back